In [1]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic

In [2]:
import gradio as gr # oh yeah!

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [8]:
# Connect to OpenAI, Anthropic and Google; comment out the Claude or Google lines if you're not using them

openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()

In [9]:
# A generic system message - no more snarky adversarial AIs!

system_message = "You are a helpful assistant"

In [10]:
# Let's wrap a call to GPT-4o-mini in a simple function

def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    return completion.choices[0].message.content

In [ ]:
# This can reveal the "training cut off", or the most recent date in the training data

message_gpt("What is today's date?")

## User Interface

In [8]:
# here's a simple function

def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [ ]:
shout("hello")

In [ ]:
# The simplicty of gradio. This might appear in "light mode" - I'll show you how to make this in dark mode later.

gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch()

In [ ]:
# Adding share=True means that it can be accessed publically
# A more permanent hosting is available using a platform called Spaces from HuggingFace, which we will touch on next week
# NOTE: Some Anti-virus software and Corporate Firewalls might not like you using share=True. If you're at work on on a work network, I suggest skip this test.

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)

In [ ]:
# Adding inbrowser=True opens up a new browser window automatically

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True, share=True)

## Forcing dark mode

Gradio appears in light mode or dark mode depending on the settings of the browser and computer. There is a way to force gradio to appear in dark mode, but Gradio recommends against this as it should be a user preference (particularly for accessibility reasons). But if you wish to force dark mode for your screens, below is how to do it.

In [ ]:
# Define this variable and then pass js=force_dark_mode when creating the Interface

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch(share=True)

In [ ]:
# Inputs and Outputs

view = gr.Interface(
    fn=shout,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never"
)
view.launch()

In [ ]:
# And now - changing the function from "shout" to "message_gpt"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never"
)
view.launch()

In [ ]:
# Let's use Markdown
# Are you wondering why it makes any difference to set system_message when it's not referred to in the code below it?
# I'm taking advantage of system_message being a global variable, used back in the message_gpt function (go take a look)
# Not a great software engineering practice, but quite sommon during Jupyter Lab R&D!

system_message = "You are a helpful assistant that responds in markdown"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

In [13]:
# Let's create a call that streams back results
# If you'd like a refresher on Generators (the "yield" keyword),
# Please take a look at the Intermediate Python notebook in week1 folder.

def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

In [15]:
def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [ ]:
view = gr.Interface(
    fn=stream_claude,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

In [17]:
def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your message:"), gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

## Summary website

In [22]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import re

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"

        for irrelevant in soup(["script", "style", "img", "input"]):
            irrelevant.decompose()

        self.text = self.extract_main_content(soup)

    def extract_main_content(self, soup):
        # 1️⃣ Tenta <article>
        article_tag = soup.find("article")
        if article_tag:
            paragraphs = article_tag.find_all("p")
            if paragraphs:
                return "\n\n".join(p.get_text(strip=True) for p in paragraphs)

        # 2️⃣ Tenta divs com classes ou IDs relacionadas a texto
        candidate_divs = soup.find_all(
            lambda tag: tag.name == "div" and (
                re.search(r"(article|content|text|post|body)", str(tag.get("class"))) or
                re.search(r"(article|content|text|post|body)", str(tag.get("id")))
            )
        )

        best_div = max(candidate_divs, key=lambda div: len(div.get_text(strip=True)), default=None)
        if best_div:
            paragraphs = best_div.find_all("p")
            if paragraphs:
                return "\n\n".join(p.get_text(strip=True) for p in paragraphs)
            else:
                return best_div.get_text(separator="\n", strip=True)

        # 3️⃣ Fallback: pega todos os <p> e filtra por tamanho mínimo
        all_paragraphs = [p.get_text(strip=True) for p in soup.find_all("p")]
        good_paragraphs = [p for p in all_paragraphs if len(p) > 50]  # só pega parágrafos relevantes

        if good_paragraphs:
            return "\n\n".join(good_paragraphs)

        # 4️⃣ Se nada encontrado
        return "No article content found."

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\n\nWebpage Contents:\n\n{self.text}\n"


In [24]:
# With massive thanks to Bill G. who noticed that a prior version of this had a bug! Now fixed.

system_message = "You are an assistant that analyzes the contents of a company website page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [25]:
# With massive thanks to Bill G. who noticed that a prior version of this had a bug! Now fixed.

system_message2 = "You are an assistant that analyzes the contents of a company website \
and creates a complete summary from it. Respond in markdown and in PT-BR."

In [26]:
def stream_gpt2(prompt):
    messages = [
        {"role": "system", "content": system_message2},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [27]:
def stream_claude2(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message2,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [28]:
def stream_brochure(company_name, url, model):
    prompt = f"Please generate a summary for {company_name} website. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    if model=="GPT":
        result = stream_gpt2(prompt)
    elif model=="Claude":
        result = stream_claude2(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Website to be summarized:"),
        gr.Textbox(label="URL including http:// or https://"),
        gr.Dropdown(["GPT", "Claude"], label="Select model")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)
view.launch()